In [ ]:
from pathlib import Path
import sys

# --- Notebook bootstrap (works from repo root or notebooks/) ---
REPO_ROOT = Path.cwd()
if not (REPO_ROOT / 'mm').exists():
    REPO_ROOT = REPO_ROOT.parent
sys.path.insert(0, str(REPO_ROOT))

DATA_ROOT = REPO_ROOT / 'data'
OUT_ROOT = REPO_ROOT / 'out'

print('REPO_ROOT:', REPO_ROOT)


# Poisson Fill Model Calibration (Template)

This notebook calibrates Poisson fill parameters for the model
$$\lambda(\delta)=A e^{-k\delta}$$
from your backtest outputs:
- `orders_<SYMBOL>.csv`
- `fills_<SYMBOL>.csv`
- `state_<SYMBOL>.csv`

It estimates empirical fill intensities by quote distance (in **ticks**), fits `log(lambda)` vs `delta`, and outputs suggested `A` and `k`.

> Assumption: you ran a baseline backtest that produces realistic fill events (typically `trade_driven`).


In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- User inputs ---
OUT_DIR = Path(os.environ.get('OUT_DIR', 'out_backtest'))
SYMBOL = os.environ.get('SYMBOL', 'BTCUSDT')

# Tick size must match what you used in the backtest
TICK_SIZE = float(os.environ.get('TICK_SIZE', '0.01'))

# Optional: exclude orders too close to mid where queue effects dominate
MIN_DELTA_TICKS = int(os.environ.get('MIN_DELTA_TICKS', '0'))

# Optional: bucket cap (avoid long tails)
MAX_DELTA_TICKS = int(os.environ.get('MAX_DELTA_TICKS', '50'))

# Minimum exposure (seconds) per bucket required to include it in the fit
MIN_EXPOSURE_S = float(os.environ.get('MIN_EXPOSURE_S', '5.0'))

orders_path = resolve_csv_path(OUT_DIR / f'orders_{SYMBOL}.csv')
fills_path  = resolve_csv_path(OUT_DIR / f'fills_{SYMBOL}.csv')
state_path  = resolve_csv_path(OUT_DIR / f'state_{SYMBOL}.csv')

print('Loading:')
print(' ', orders_path)
print(' ', fills_path)
print(' ', state_path)

orders = pd.read_csv(orders_path)
fills  = pd.read_csv(fills_path)
state  = pd.read_csv(state_path)

orders.head(), fills.head(), state.head()

from pathlib import Path

def resolve_csv_path(p: Path) -> Path:
    """Return p if exists; otherwise try p+'.gz'. Accepts either .csv or .csv.gz."""
    if p.exists():
        return p
    gz = Path(str(p) + '.gz')
    if gz.exists():
        return gz
    # also allow replacing suffix if user provided .csv.gz already
    if p.suffix == '.gz' and p.exists():
        return p
    raise FileNotFoundError(f'File not found: {p} (or {gz})')




Loading:
  out_backtest/orders_BTCUSDT.csv
  out_backtest/fills_BTCUSDT.csv
  out_backtest/state_BTCUSDT.csv


(         recv_ms                              order_id      action  \
 0  1766127600134  8bf023bf-2502-4ae6-bb37-00544b009adc       PLACE   
 1  1766127603417  8bf023bf-2502-4ae6-bb37-00544b009adc  CANCEL_REQ   
 2  1766127603417  61b523bc-f9fe-4637-baa1-83b6152fe97f       PLACE   
 3  1766127603517  8bf023bf-2502-4ae6-bb37-00544b009adc  CANCEL_ACK   
 4  1766127604594  61b523bc-f9fe-4637-baa1-83b6152fe97f        FILL   
 
            status side    price    qty  remaining_qty  active_recv_ms  \
 0            OPEN  BUY  87482.7  0.001        0.00100   1766127600184   
 1  CANCEL_PENDING  BUY  87482.7  0.001        0.00100   1766127600184   
 2            OPEN  BUY  87483.3  0.001        0.00100   1766127603467   
 3       CANCELLED  BUY  87482.7  0.001        0.00100   1766127600184   
 4            OPEN  BUY  87483.3  0.001        0.00094   1766127603467   
 
    expire_recv_ms  cancel_req_ms  cancel_effective_ms  reject_reason  \
 0             NaN            NaN                  Na

## Build per-order exposure records

For each placed order we compute:
- `active_ms` (when it becomes fill-eligible)
- `end_ms` = min(first fill time, cancel effective time, end of run)
- `exposure_s` = (end_ms - active_ms)/1000
- `delta_ticks` = distance from mid at activation time

Then we aggregate by `delta_ticks` bucket:
- total exposure time
- number of fill events (or filled qty)


In [ ]:
# Ensure correct dtypes
for col in ['recv_ms','active_recv_ms','expire_recv_ms','cancel_effective_ms']:
    if col in orders.columns:
        orders[col] = pd.to_numeric(orders[col], errors='coerce')

fills['recv_ms'] = pd.to_numeric(fills['recv_ms'], errors='coerce')
state['recv_ms'] = pd.to_numeric(state['recv_ms'], errors='coerce')

# Consider only PLACE events as order creations
placed = orders[orders['action'] == 'PLACE'].copy()

# Drop rejected placements if present
if 'status' in placed.columns:
    placed = placed[placed['status'] != 'REJECTED'].copy()

# Active time: if missing, fall back to recv_ms
placed['active_ms'] = placed['active_recv_ms'].fillna(placed['recv_ms']).astype('int64')

# Determine an end-of-run timestamp
end_run_ms = int(max(state['recv_ms'].max(), orders['recv_ms'].max(), fills['recv_ms'].max()))
print('End of run ms:', end_run_ms)

# First fill time per order
first_fill_ms = fills.groupby('order_id', as_index=False)['recv_ms'].min().rename(columns={'recv_ms':'first_fill_ms'})

# Cancel effective time per order (from CANCEL_REQ rows)
cancel_req = orders[orders['action'] == 'CANCEL_REQ'].copy()
if 'cancel_effective_ms' in cancel_req.columns:
    cancel_eff = cancel_req.groupby('order_id', as_index=False)['cancel_effective_ms'].min().rename(columns={'cancel_effective_ms':'cancel_eff_ms'})
else:
    cancel_eff = pd.DataFrame(columns=['order_id','cancel_eff_ms'])

# Merge into placed
placed = placed.merge(first_fill_ms, on='order_id', how='left')
placed = placed.merge(cancel_eff, on='order_id', how='left')

# Compute end time = min(first fill, cancel effective, end)
placed['end_ms'] = end_run_ms
mask = placed['first_fill_ms'].notna()
placed.loc[mask, 'end_ms'] = np.minimum(placed.loc[mask, 'end_ms'], placed.loc[mask, 'first_fill_ms'])
mask = placed['cancel_eff_ms'].notna()
placed.loc[mask, 'end_ms'] = np.minimum(placed.loc[mask, 'end_ms'], placed.loc[mask, 'cancel_eff_ms'])

# Exposure time in seconds (clip at >=0)
placed['exposure_s'] = (placed['end_ms'] - placed['active_ms']).clip(lower=0) / 1000.0

# Mid at activation time: merge_asof
state_sorted = state[['recv_ms','mid']].sort_values('recv_ms')
placed_sorted = placed.sort_values('active_ms')
mid_at_active = pd.merge_asof(
    placed_sorted,
    state_sorted,
    left_on='active_ms',
    right_on='recv_ms',
    direction='backward'
).rename(columns={'mid':'mid_at_active'})

# Distance in ticks
mid_at_active['delta_ticks'] = (np.abs(mid_at_active['price'] - mid_at_active['mid_at_active']) / TICK_SIZE)
mid_at_active['delta_bucket'] = np.floor(mid_at_active['delta_ticks']).astype(int)

# Filter buckets
mid_at_active = mid_at_active[(mid_at_active['delta_bucket'] >= MIN_DELTA_TICKS) & (mid_at_active['delta_bucket'] <= MAX_DELTA_TICKS)].copy()

mid_at_active[['order_id','side','price','mid_at_active','delta_ticks','delta_bucket','exposure_s']].head(10)


## Empirical intensity by distance bucket

Two common choices:
- **Event intensity**: number of fill events per second (closer to the AS arrival-rate assumption)
- **Size intensity**: filled quantity per second

This template computes **event intensity** by default.

In [ ]:
# Fill event counts and filled qty per order
fill_event_counts = fills.groupby('order_id').size().rename('fill_events').reset_index()
filled_qty = fills.groupby('order_id', as_index=False)['qty'].sum().rename(columns={'qty':'filled_qty'})

df = mid_at_active.merge(fill_event_counts, on='order_id', how='left').merge(filled_qty, on='order_id', how='left')
df['fill_events'] = df['fill_events'].fillna(0).astype(int)
df['filled_qty'] = df['filled_qty'].fillna(0.0)

# Aggregate by delta bucket
agg = df.groupby('delta_bucket', as_index=False).agg(
    exposure_s=('exposure_s','sum'),
    fill_events=('fill_events','sum'),
    filled_qty=('filled_qty','sum'),
    n_orders=('order_id','count')
)

agg['lambda_events_per_s'] = agg['fill_events'] / agg['exposure_s'].replace(0, np.nan)
agg['lambda_qty_per_s'] = agg['filled_qty'] / agg['exposure_s'].replace(0, np.nan)

fit_df = agg[(agg['exposure_s'] >= MIN_EXPOSURE_S) & (agg['lambda_events_per_s'] > 0)].copy()

agg.head(15), fit_df.head(15)


In [ ]:
plt.figure()
plt.plot(agg['delta_bucket'], agg['lambda_events_per_s'], marker='o')
plt.xlabel('delta (ticks)')
plt.ylabel('empirical lambda (fill events / second)')
plt.title('Empirical Fill Intensity by Quote Distance')
plt.yscale('log')
plt.grid(True)
plt.show()


## Fit `log(lambda)` vs `delta`

We fit:
$$\log \lambda(\delta) = \log A - k\delta$$

Weighted by exposure time (more exposure ⇒ more reliable estimate).

In [ ]:
if len(fit_df) < 2:
    raise RuntimeError('Not enough buckets with positive intensity and sufficient exposure to fit. Try lowering MIN_EXPOSURE_S or increasing MAX_DELTA_TICKS.')

x = fit_df['delta_bucket'].to_numpy(dtype=float)
y = np.log(fit_df['lambda_events_per_s'].to_numpy(dtype=float))
w = fit_df['exposure_s'].to_numpy(dtype=float)

# Weighted linear fit: y = b0 + b1*x, where b1 = -k and b0 = log(A)
b1, b0 = np.polyfit(x, y, deg=1, w=w)
k_hat = -b1
A_hat = float(np.exp(b0))

print('Fitted parameters (event intensity):')
print('  A =', A_hat)
print('  k =', k_hat)

x_line = np.linspace(x.min(), x.max(), 100)
y_line = b0 + b1 * x_line

plt.figure()
plt.scatter(x, y, s=40)
plt.plot(x_line, y_line)
plt.xlabel('delta (ticks)')
plt.ylabel('log(lambda)')
plt.title('Log-Intensity Fit: log(lambda)=log(A)-k*delta')
plt.grid(True)
plt.show()


## Suggested `FILL_PARAMS_JSON`

Set `dt_ms` to a reasonably small step (commonly 50–200 ms). If unsure, start with 100 ms.

In [ ]:
dt_ms = int(os.environ.get('POISSON_DT_MS', '100'))
params = {'A': float(A_hat), 'k': float(k_hat), 'dt_ms': dt_ms}
print('FILL_MODEL=poisson')
print('FILL_PARAMS_JSON=' + json.dumps(params))


## Notes / interpretation

- `A` is the baseline fill-event rate near the mid (units: events/sec).
- `k` controls how quickly the fill rate decays as you quote wider.
- If you see implausibly large `A`, consider excluding very small deltas (`MIN_DELTA_TICKS=1` or `2`) and re-fitting.
